### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성청년SW·AI아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

### **Objectives**

1. **과제명**: MLP 구현

2. **핵심 주제**
  - `scikit-learn Pipeline` 및 `GridSearchCV`를 이용한 모델 파이프라인 구축  
  - PyTorch Lightning DataModule을 활용한 데이터 전처리 및 로딩 관리  
  - LightningModule을 통한 MLP 네트워크 구현 및 학습/평가 자동화

3. **학습 목표**
  - `sklearn Pipeline`을 설계하여 데이터 전처리부터 모델 학습까지 일괄 처리할 수 있다.  
  - `GridSearchCV`를 활용하여 하이퍼파라미터 탐색 및 최적화를 수행할 수 있다.  
  - PyTorch Lightning DataModule을 정의하여 데이터 로딩 과정을 구조화할 수 있다.  
  - `LightningModule`을 구현하여 MLP 모델의 training/validation 루프를 관리할 수 있다.  
  - 콜백을 이용해 체크포인트 저장 및 EarlyStopping 설정으로 학습 과정을 제어할 수 있다.  

4. **학습 개념**
  - `Pipeline`: 전처리와 학습 단계를 연결해 일괄 실행하는 scikit-learn 구성 요소  
  - `GridSearchCV`: 교차 검증을 통해 최적의 하이퍼파라미터 조합을 탐색하는 기법  
  - `DataModule`: PyTorch Lightning에서 데이터 준비와 DataLoader 관리를 캡슐화하는 모듈  
  - `LightningModule`: 모델 정의, 학습/검증 스텝, 옵티마이저 설정 등을 추상화한 PyTorch Lightning 컴포넌트  
  - `MLP`: 다층 퍼셉트론 구조로 구성된 인공 신경망  

5. **학습 방향**
  - scikit-learn Pipeline과 GridSearchCV를 실습해 전처리·학습·튜닝 파이프라인을 설계한다.  
  - torchvision의 FashionMNIST 데이터를 DataModule로 캡슐화하여 로딩과 전처리를 일원화한다.  
  - LightningModule을 통해 MLP 모델 구조와 학습/검증 루프를 구현하고 Trainer로 실행한다.  
  - ModelCheckpoint, EarlyStopping, WandbLogger 등 콜백과 로거를 적용해 모델 저장, 학습 중단, 실험 관리를 수행한다.  

6. **데이터셋 개요 및 저작권 정보**
  - 데이터셋 명: FashionMNIST  
  - 데이터셋 개요: 10개 의류 카테고리(티셔츠, 바지 등)의 28×28 흑백 이미지 학습 60,000장, 테스트 10,000장으로 구성  
  - 데이터셋 저작권: Zalando SE 소유, 연구 및 교육 목적으로 공개. torchvision 라이브러리를 통해 제공  

### **Prerequisites**
```
numpy>=1.26
pandas>=2.0
scikit-learn>=1.4
seaborn>=0.12
torch>=2.2
matplotlib>=3.8
pytorch-lightning==2.5.3
```


# `scikit-learn`을 추가로 응용해보기
**학습 목표**  
  - sklearn Pipeline을 구축하여 데이터 전처리와 모델 학습 단계를 일관되게 처리한다.  
  - `GridSearchCV`를 활용해 다양한 모델과 하이퍼파라미터 조합을 탐색하고 최적 모델을 선정한다.  
  - 교차 검증 결과와 평가 지표(MSE, R²)를 통해 회귀 모델 성능을 검증한다.  

**학습 개념**  
  - `Pipeline`: 전처리(`StandardScaler`)와 모델(`LinearRegression`, `Ridge`, `Lasso`, `MLPRegressor`)을 순차적으로 연결해 자동 실행한다.  
  - `GridSearchCV`: 파이프라인 내부 모델 파라미터를 dict 형태로 정의해 교차 검증으로 최적 파라미터를 찾는다.  

**진행하는 실습 요약**  
  1. seaborn의 mpg 데이터셋을 로드해 결측치 제거, 원-핫 인코딩, 타깃 분리, train/test 분할을 수행한다.  
  2. `StandardScaler`+`LinearRegression` 파이프라인으로 기본 회귀 모델을 실습하고 결과(MSE, R²) 확인.  
  3. `Ridge`, `Lasso`, `MLPRegressor`를 각각 `Pipeline`으로 정의해 딕셔너리에 저장한다.  
  4. `GridSearchCV`로 각 파이프라인의 하이퍼파라미터 탐색 후 최적 모델을 선정하고 테스트 세트에서 성능을 평가한다.  

이번 과제에서는 이론시간에 다룬 여러 모델들을 직접 `scikit-learn`에서 불러서 학습해봅시다. 여러 초매개변수도 바꾸고 모델도 바꿔보면서 어떤 경우에 성능이 더 잘 나오는지 확인해보는 간단한 코드 실습을 해봅시다.

오늘은 `sklearn.pipeline.Pipeline`을 이용해서 데이터 처리과정을 하나의 파이프로 넣어서 간편하게 처리하는 방법과 optuna가 아닌 `sklearn.model_selection.GridSearchCV`를 통해 초매개변수를 탐색하는 방법에 대해서도 배워봅니다. 회귀와 분류과제 각각 진행해봅시다.

### 🧑‍💻 **과제**: 회귀 과제 다시 수행해보기

`mpg` 데이터셋을 다양한 회귀 모델을 통해 학습해봅시다. 우선 데이터를 전처리해줍시다.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch

RANDOM_STATE = 42
df = sns.load_dataset('mpg').dropna()

# 아래는 난수를 고정하는 함수입니다.
# 재현성을 위해 구현되어 있습니다.
import random
def set_seed(seed: int = 42):
    random.seed(seed)                       # Python 기본 random 시드 고정
    np.random.seed(seed)                    # NumPy 난수 시드 고정
    torch.manual_seed(seed)                 # PyTorch CPU 난수 시드 고정
    torch.cuda.manual_seed_all(seed)        # GPU(CUDA) 난수 시드 고정
    torch.backends.cudnn.deterministic = True  # 연산 결과를 항상 동일하게 (Deterministic 모드)
    torch.backends.cudnn.benchmark = False     # 연산 최적화 비활성화 (결과 일관성 유지)

set_seed(seed=RANDOM_STATE)

In [2]:
# TODO
# 1. 데이터셋 불러오기
# 지난 실습에서 `name` column은 사용하지 않기로 했습니다. 이 점 유의해서 X, y 생성해주세요
# 생성한 X, y 기반으로 X_train, X_test, y_train, y_test 생성해주세요.
# 테스트데이터는 전체 학습데이터의 20%로 설정해주세요.
from sklearn.model_selection import train_test_split
X = df.drop(columns=['mpg', 'name'])
X = pd.get_dummies(X, drop_first=True) # 범주형 변수 원-핫 인코딩 (첫 카테고리 제거)
y = df['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

🪈 `sklearn.pipeline.Pipeline`

아래는 공식 API Reference에서 명시된 Pipeline의 예시코드입니다.
```python
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe.fit(X_train, y_train).score(X_test, y_test)
# An estimator's parameter can be set using '__' syntax
pipe.set_params(svc__C=10).fit(X_train, y_train).score(X_test, y_test)
```

잘 읽어보면 두 가지를 알 수 있습니다.
- 데이터 전처리 모듈을 넣을 수 있습니다. 데이터누수를 알아서 방지해줍니다.
- 데이터 누수: 모델이 학습 중에 원래 알 수 없어야 할 테스트 데이터나 미래의 정보를 미리 접해 잘못된 성능 향상을 보이는 현상
- 모델도 넣을 수 있습니다.
- 이미 선언된 파이프라인 내에서 속성을 변경할 수 있습니다.

해당 내용을 통해 아래 과제를 수행해봅시다.

In [3]:
# TODO
# 2. StandardScaler와 LinearRegression을 Pipeline으로 구성해주세요.
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

# 3. 학습데이터에 대해 학습을 진행해주고, 회귀성능을 평가해주세요
from sklearn.metrics import mean_squared_error, r2_score
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(f"MSE: {mean_squared_error(y_true=y_test, y_pred=y_pred):.4f}")
print(f"R2: {r2_score(y_true=y_test, y_pred=y_pred):.4f}")

MSE: 10.6023
R2: 0.7923


성공적으로 `Pipeline`으로 많은 과정을 손쉽게 해결하였습니다!

🔍 `sklearn.model_selection.GridSearchCV`

아래는 공식 API Reference에서 명시된 GridSearchCV의 예시코드입니다.
```python
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)
sorted(clf.cv_results_.keys())
```

사용방법은 `GridSearchCV`에 estimator와 탐색대상인 parameters를 dict 형태로 제공합니다. `scoring`을 넣어주면 해당 지표에 따라 최적화를 진행하여 초매개변수를 찾아줍니다.

우리는 Pipeline과 GridSearchCV를 모두 사용하여 회귀과제를 제일 잘 수행하는 천하제일회귀모델을 찾아줍시다.


In [4]:
# 3. 여러 모델의 Pipelines를 정의해줍시다.
# dict 형태로 제가 key만 넣어두었습니다. value들은 모두 Pipeline으로 만들어주세요.
from sklearn.linear_model import Ridge, Lasso
from sklearn.neural_network import MLPRegressor

pipelines = {
    "Ridge": Pipeline([
        ('scaler', StandardScaler()),
        ('model', Ridge())
    ]),
    "Lasso": Pipeline([
        ('scaler', StandardScaler()),
        ('model', Lasso())
    ]),
    "MLPRegressor": Pipeline([
        ('scaler', StandardScaler()),
        ('model', MLPRegressor(max_iter=500, random_state=RANDOM_STATE))
    ])
}

# 4. 하이퍼파라미터 그리드 정의
# 위에 `pipelines`에 있는 모델들 별로 hyperparameter를 찾아줍니다.
# ⚠️ 제가 GridSearchCV의 API Reference를 들고온 이유가 있습니다.
# Search space key를 어떻게 정의해줘야할까요?
param_grids = {
    "Ridge": {'model__alpha': [0.1, 1.0, 10.0]},
    "Lasso": {'model__alpha': [0.01, 0.1, 1.0]},
    "MLPRegressor": {
        'model__hidden_layer_sizes': [(64,), (64, 32)],
        'model__alpha': [0.0001, 0.001]
        # Scikit-learn의 MLP는 층의 개수와 노드 수만 지정할 수 있음
    }
}

# 5. 천하제일회귀모델 찾아주기
# 아래 루프를 돌리면서 GridSearchCV를 수행해주세요.
from sklearn.model_selection import GridSearchCV
results = []
for name, pipeline in pipelines.items():
    # 모델 별로 GridSearchCV를 수행해줍니다.
    # Fold 수나 scoring은 여러분이 자유롭게 선택해서 제일 좋은 회귀모델을 찾아주세요.
    # GridSearchCV의 결과를 통해 모델 별로 제일 잘 학습된 estimator를 찾아 `best_model`에 할당해주세요.
    # Grid Search가 어떻게 수행되고 있는지 확인해보려면 어떻게 해야할까요? hint: verbose
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grids[name],
        cv=3,
        scoring='r2',
        verbose=2,
    )
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_

    # `best_model`로 `y_pred`를 예측해주세요.
    # 그 후에 MSE, R²를 results에 기록해주세요.
    y_pred = best_model.predict(X_test)

    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    r2  = r2_score(y_true=y_test, y_pred=y_pred)
    results.append({
        "Model":       name,
        "Best Params": grid.best_params_,
        "MSE":         mse,
        "R²":          r2,
    })

# 결과 출력: 어떤 모델의 성능이 제일 좋은가요?
pd.DataFrame(results).sort_values(by="MSE", ascending=True)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ...................................model__alpha=0.1; total time=   0.1s
[CV] END ...................................model__alpha=0.1; total time=   0.0s
[CV] END ...................................model__alpha=0.1; total time=   0.0s
[CV] END ...................................model__alpha=1.0; total time=   0.0s
[CV] END ...................................model__alpha=1.0; total time=   0.0s
[CV] END ...................................model__alpha=1.0; total time=   0.0s
[CV] END ..................................model__alpha=10.0; total time=   0.0s
[CV] END ..................................model__alpha=10.0; total time=   0.0s
[CV] END ..................................model__alpha=10.0; total time=   0.0s
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ..................................model__alpha=0.01; total time=   0.0s
[CV] END ..................................model__alpha=0.01; total time

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END model__alpha=0.0001, model__hidden_layer_sizes=(64,); total time=   1.2s


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END model__alpha=0.0001, model__hidden_layer_sizes=(64,); total time=   0.9s


...

[CV] END model__alpha=0.001, model__hidden_layer_sizes=(64,); total time=   0.5s


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END model__alpha=0.001, model__hidden_layer_sizes=(64,); total time=   0.4s
[CV] END model__alpha=0.001, model__hidden_layer_sizes=(64, 32); total time=   0.6s
[CV] END model__alpha=0.001, model__hidden_layer_sizes=(64, 32); total time=   0.5s
[CV] END model__alpha=0.001, model__hidden_layer_sizes=(64, 32); total time=   0.5s


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,Model,Best Params,MSE,R²
2,MLPRegressor,"{'model__alpha': 0.0001, 'model__hidden_layer_...",7.037229,0.862125
0,Ridge,{'model__alpha': 1.0},10.659664,0.791153
1,Lasso,{'model__alpha': 0.1},10.873755,0.786959


### 🧑‍💻 **과제**: 분류 과제 다시 수행해보기

이번에는 지난 시간에 사용한 숫자필기 데이터셋을 `sklearn`에 있는 모델들로 학습해봅시다. MLP의 경우 MLPClassifier를 활용합시다.



In [5]:
# 1. 데이터 로드 및 전처리
# load_digits 데이터셋 로드
from sklearn.datasets import load_digits
digits = load_digits()
X, y = digits.data, digits.target

# TODO: 데이터 불러오기
# train/test 분할
# 테스트데이터는 전체 데이터의 20%로 설정해주세요.
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

In [6]:
# TODO:
# 2. 여러 모델의 Pipelines를 정의해줍시다.
# 다음 모델들을 비교해주세요: LogisticRegression, DecisionTree, Bagging, RandomForest, AdaBoost, MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

pipelines = {
    "LogisticRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model",  LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
    ]),
    "DecisionTree": Pipeline([
        ("scaler", StandardScaler()),
        ("model",  DecisionTreeClassifier(random_state=RANDOM_STATE))
    ]),
    "Bagging": Pipeline([
        ("scaler", StandardScaler()),
        ("model",  BaggingClassifier(random_state=RANDOM_STATE))
    ]),
    "RandomForest": Pipeline([
        ("scaler", StandardScaler()),
        ("model",  RandomForestClassifier(random_state=RANDOM_STATE))
    ]),
    "AdaBoost": Pipeline([
        ("scaler", StandardScaler()),
        ("model",  AdaBoostClassifier(random_state=RANDOM_STATE))
    ]),
    "MLPClassifier": Pipeline([
        ("scaler", StandardScaler()),
        ("model",  MLPClassifier(max_iter=500, random_state=RANDOM_STATE))
    ]),
}

# TODO:
# 3. 하이퍼파라미터 그리드 정의
# 위에 `pipelines`에 있는 모델들 별로 hyperparameter를 찾아줍니다.
param_grids = {
    "LogisticRegression": {
        "model__C": [0.01, 0.1, 1.0, 10.0]
    },
    "DecisionTree": {
        "model__max_depth": [None, 5, 10, 20],
        "model__min_samples_split": [2, 5, 10]
    },
    "Bagging": {
        "model__n_estimators": [10, 50, 100],
        "model__max_samples": [0.5, 1.0]
    },
    "RandomForest": {
        "model__n_estimators": [50, 100, 200],
        "model__max_depth": [None, 5, 10],
        "model__min_samples_split": [2, 5]
    },
    "AdaBoost": {
        "model__n_estimators": [50, 100, 200],
        "model__learning_rate": [0.01, 0.1, 1.0]
    },
    "MLPClassifier": {
        "model__hidden_layer_sizes": [(64,), (128, 64)],
        "model__alpha": [0.0001, 0.001],
        "model__learning_rate_init": [0.001, 0.01]
    }
}

# TODO:
# 4. 천하제일분류모델 찾아주기
# 아래 루프를 돌리면서 GridSearchCV를 수행해주세요.
from sklearn.metrics import accuracy_score, f1_score

results = []
for name, pipe in pipelines.items():
    # 모델 별로 GridSearchCV를 수행해줍니다.
    # Fold 수나 scoring은 여러분이 자유롭게 선택해서 제일 좋은 회귀모델을 찾아주세요.
    # GridSearchCV의 결과를 통해 모델 별로 제일 잘 학습된 estimator를 찾아 `best_model`에 할당해주세요.
    # 마찬가지로 Grid Search의 내부 동작을 확인할 수 있도록 argument를 조절해주세요.
    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grids[name],
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        verbose=2,
    )
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_

    # `best_model`로 `y_pred`를 예측해주세요.
    # 그 후에 ACC, F1-score를 results에 기록해주세요.
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_true=y_test, y_pred=y_pred)
    f1  = f1_score(y_true=y_test, y_pred=y_pred, average='macro')

    results.append({
        "Model":       name,
        "Best Params": grid.best_params_,
        "Accuracy":    acc,
        "F1 Score":    f1,
    })

# 5. 결과 테이블 출력
pd.DataFrame(results).sort_values(by="Accuracy", ascending=False)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 18 candidates, totalling 54 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits


,Model,Best Params,Accuracy,F1 Score
5,MLPClassifier,"{'model__alpha': 0.0001, 'model__hidden_layer_...",0.980556,0.980266
0,LogisticRegression,{'model__C': 1.0},0.972222,0.971940
3,RandomForest,"{'model__max_depth': 10, 'model__min_samples_s...",0.963889,0.963541
2,Bagging,"{'model__max_samples': 0.5, 'model__n_estimato...",0.933333,0.932732
4,AdaBoost,"{'model__learning_rate': 1.0, 'model__n_estima...",0.863889,0.865207
1,DecisionTree,"{'model__max_depth': None, 'model__min_samples...",0.816667,0.814841


# ⚡️ PyTorch Lightning이란?

- PyTorch Lightning은 PyTorch의 훈련 절차(학습 루프, 로깅, GPU 관리 등)를 깔끔하게 모듈화해, 모델 설계에만 집중할 수 있게 해주는 고수준 프레임워크

**학습 목표**  
  - `LightningDataModule`로 `FashionMNIST` 데이터 로딩과 전처리를 모듈화한다.  
  - `LightningModule`을 활용해 MLP 기반 이미지 분류 모델의 학습/검증 루프를 추상화한다.  
  - `Trainer`에서 `ModelCheckpoint`, `EarlyStopping`, `WandbLogger` 등 콜백을 적용해 학습 과정을 관리한다.  

**학습 개념**  
  - `LightningDataModule`: `prepare_data`, `setup`, `train_dataloader`, `val_dataloader` 메서드로 데이터 파이프라인 구조화  
  - `LightningModule`: `forward`, `training_step`, `validation_step`, `configure_optimizers`로 모델 학습 로직 캡슐화  
  - `Trainer`: `max_epochs`, `accelerator`, `callbacks`, `logger` 옵션으로 학습 실행 환경 제어  
  - 콜백(Callbacks) 및 로거(Loggers):  
    - `ModelCheckpoint`: 최적의 검증 성능 시점 모델 자동 저장  
    - `EarlyStopping`: 지정 에폭 수 동안 성능 향상 없을 시 학습 조기 중단  
    - `WandbLogger`: 실험 메트릭과 파라미터를 Weights & Biases에 기록  

**진행하는 실습 요약**  
  1. FashionMNIST 데이터를 `ToTensor`, `Normalize`로 전처리하고 DataModule에 캡슐화한다.  
  2. MLPClassifier `LightningModule`을 구현해 2개 히든 레이어, 드롭아웃, CrossEntropyLoss 기반 분류 모델을 정의한다.  
  3. `pl.Trainer`를 사용해 max_epochs=10, accelerator='auto', callbacks와 logger를 설정하여 학습을 실행한다.  
  4. 최적화된 체크포인트 저장, 조기 중단을 구현한다.  


지난 시간에 PyTorch를 처음으로 다뤄보았습니다. 과거에는 PyTorch 하나만 가지고 모든 개발을 다했어야 하는데, 지금은 반복적인 코드를 단순화시켜둔 많은 프레임워크가 생겼습니다. 대표적인 것이 PyTorch Lightning입니다.

PyTorch가 기본 블록만 있는 라이브러리라면, PyTorch Lightning은 이 블록들을 조금 더 유기적으로 잘 조합해둔 구조화된 프레임워크입니다. 예를 들어, 모델 정의, 학습 루프, 평가, 로깅, GPU 설정 등 반복적으로 작성하던 코드를 하나의 클래스로 깔끔하게 정리할 수 있습니다.

이 덕분에 우리는 모델의 핵심 로직에만 집중할 수 있고, 실험 관리나 확장도 훨씬 쉬워집니다. 특히 학습 코드가 길어질수록 Lightning의 장점은 더욱 두드러집니다. 이번 실습에서는 PyTorch Lightning을 활용해 MLP 모델을 FashionMNIST 데이터셋에 적용해보고, 기존 PyTorch 코드와 어떤 차이가 있는지 직접 체감해보는 시간을 가져보겠습니다.

### 📖 **자습**: PyTorch-Lightning으로 FashionMNIST 학습하기

크게 3가지 순서로 구성됩니다.
1. 데이터셋을 정의하는 부분: 사실 필수적이지는 않습니다. 우리가 알던 `torch.utils.data.DataLoader`를 그대로 활용해도 무방합니다.
2. 모델을 정의하는 부분: `torch.nn.Module` 대신 `pl.LightningModule`을 사용합니다.
3. 학습을 진행하는 부분: `pl.Trainer`가 학습에 사용하는 많은 코드를 대신해줍니다.

아래 코드를 자세히 읽어보고 차이점을 확인해주세요. 오늘의 과제는 제가 작성해드린 코드에 추가로 Callback을 추가하는 것이기 때문에 아래 코드를 잘 이해하셔야합니다.


In [8]:
!pip install pytorch-lightning==2.5.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.2/828.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 48.8 MB/s eta 0:00:00


In [9]:
import pytorch_lightning as pl
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# PyTorch Lightning의 DataModule은 데이터 관련 로직을 하나의 클래스로 깔끔하게 분리할 수 있게 해줌
# PyTorch에서는 데이터셋 로딩, 전처리, DataLoader 정의를 main 코드에 직접 작성해야 했지만,
# Lightning에서는 이를 모듈화하여 재사용성과 가독성을 높임
# 이렇게 작성하면 지난 시간에 반복적으로 작성한 train_loader, valid_loader를 별도의 변수로 관리하지 않아도 됨
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size

        # transform 정의도 클래스 내부에 포함시켜 관리 가능
        # PyTorch에서는 transform을 따로 정의하고 전달해야 했지만,
        # Lightning에서는 이처럼 모듈 내부에서 일관되게 관리 가능
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    # prepare_data는 데이터 다운로드 등 한 번만 수행되는 작업을 정의
    # PyTorch에서는 이런 작업도 main 코드에 직접 작성해야 했지만,
    # Lightning은 분리된 메서드로 관리하여 코드 구조를 명확히 함
    def prepare_data(self):
        datasets.FashionMNIST(root='./data', train=True, download=True)
        datasets.FashionMNIST(root='./data', train=False, download=True)

    # setup은 train/val/test 데이터셋을 정의하는 단계
    # stage 인자를 통해 학습/테스트 단계별로 다른 설정도 가능
    # PyTorch에서는 이런 분기 처리를 직접 구현해야 했지만,
    # Lightning은 이를 표준화된 방식으로 제공
    def setup(self, stage=None):
        self.train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=self.transform)
        self.val_dataset = datasets.FashionMNIST(root='./data', train=False, transform=self.transform)

    # train_dataloader는 학습용 DataLoader 반환
    # PyTorch에서는 직접 DataLoader를 만들고 전달해야 했지만,
    # Lightning은 Trainer가 자동으로 이 메서드를 호출해 사용함
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    # val_dataloader는 검증용 DataLoader 반환
    # PyTorch에서는 검증 루프를 직접 작성해야 했지만,
    # Lightning은 validation_step과 함께 자동으로 처리 가능
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    # predict_dataloader 검증용 DataLoader 반환
    # 추후에 `trainer.predict`를 통해 자세한 검증결과 확인을 위해 구현
    # Lightning은 predict_step 함께 자동으로 처리 가능
    def predict_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

In [11]:
import torch
from torch import nn
import torch.nn.functional as F

# LightningModule은 모델 정의, 학습/검증 로직, 옵티마이저 설정까지 하나의 클래스에 통합
# PyTorch에서는 이 모든 것을 main 코드에서 따로 관리해야 했지만,
# Lightning은 이를 구조화하여 코드 재사용성과 가독성을 높임
# nn.Module을 상속 받는 것이 아니라 pl.LightningModule을 상속 받습니다.
class MLPClassifier(pl.LightningModule):
    def __init__(self, input_dim=28*28, hidden_dims=(128, 64), dropout=0.2, num_classes=10):
        # PyTorch를 사용할 때와 마찬가지로 반드시 작성해줘야합니다.
        super().__init__()

        # 모델 구조 정의: Sequential로 간단하게 구성
        # PyTorch에서는 forward 함수 내부에 직접 레이어를 호출해야 했지만,
        # Lightning에서는 이처럼 깔끔하게 정리 가능
        h1, h2 = hidden_dims
        self.model = nn.Sequential(
            nn.Flatten(),  # 입력 이미지(28x28)를 1D 벡터로 변환
            nn.Linear(input_dim, h1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h2, num_classes)
        )

    # forward 함수는 모델의 순전파 정의
    # Lightning은 이 함수를 내부적으로 호출하므로 사용자는 따로 신경 쓸 필요 없음
    def forward(self, x):
        return self.model(x)

    # training_step은 학습 루프의 한 스텝을 정의 (배치단위)
    # PyTorch에서는 epoch 루프, 배치 루프, loss 계산, optimizer step 등을 직접 작성해야 했지만,
    # Lightning은 이 메서드만 정의하면 Trainer가 자동으로 반복 처리
    # 지난 실습에 제작한 `train_one_epoch` 함수와 동일한 기능을 합니다.
    # return 값에는 `.backward`를 호출할 수 있는 텐서가 반환되거나
    # dict로 반환될 경우 `loss`라는 key가 존재해야 합니다.
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()

        # log_dict을 통해 loss와 accuracy를 로깅
        # PyTorch에서는 TensorBoard나 tqdm 등을 직접 설정해야 했지만,
        # Lightning은 자동 로깅 및 프로그레스 바 지원
        result = {"loss": loss, "acc": acc}
        self.log_dict(result, prog_bar=True)
        return loss

    # validation_step은 검증 루프의 한 스텝을 정의
    # PyTorch에서는 검증 루프도 직접 작성해야 했지만,
    # Lightning은 이 메서드만 정의하면 자동으로 validation_epoch_end까지 처리
    # 지난 실습에 제작한 `evaluate`와 같은 기능입니다.
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        result = {"val_loss": loss, "val_acc": acc}
        self.log_dict(result, prog_bar=True)

    # configure_optimizers는 옵티마이저 설정
    # PyTorch에서는 optimizer를 따로 정의하고 학습 루프에 넣어야 했지만,
    # Lightning은 이 메서드에서 반환만 하면 Trainer가 자동으로 사용
    # 매번 optimizer를 불러서 zero_grad, step을 수행하지 않아도 알아서 작동함
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [12]:
# 아래 코드를 수행하면 위에서 제작한 모듈들이 유기적으로 작동하여 학습이 진행됩니다.
# `trainer`는 학습에서 중요한 여러 세팅값들을 받습니다.
# API Reference를 꼭 확인해주세요.
# https://lightning.ai/docs/pytorch/stable/common/trainer.html#trainer-class-api
data_module = FashionMNISTDataModule(batch_size=1024)
model = MLPClassifier()
trainer = pl.Trainer(max_epochs=5, accelerator="auto")
trainer.fit(model, data_module)
trainer.validate(model, data_module)

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
100%|██████████| 26.4M/26.4M [00:02<00:00, 12.4MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 210kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.92MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.27MB/s]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 109 K  | train
---------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │     0.852400004863739     │
│         val_loss          │    0.40553852915763855    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.40553852915763855, 'val_acc': 0.852400004863739}]

### 🧑‍💻 **과제**: `pl.LightningModule`에 구성요소 추가하기

지난 번에 제작한 코드와 비교했을 때 많이 간결해졌죠? 여기에 더 많은 기능들을 구현할 수 있습니다.
1. 학습 전과 후: 현재 `training_step`이랑 `validation_step`만 구현했는데, 추가로 학습 전이나 후, 검증 전이나 후에 추가로 액션을 취할 수 있습니다. 하나의 epoch가 끝나고나서 epoch의 평균 손실이나 정확도 같은 것을 계산하고 싶을 때는 어떻게해야할까요? 그럴 때는 `training_epoch_end`, `validation_epoch_end` 메소드를 구현하면 됩니다.
2. `wandb logger`: 지난 실습 말미에 `wandb` 라이브러리를 소개했습니다. `pl.LightningModule`에서 사용하는 `log_dict` 함수를 `wnadb`에 바로 연동시킬 수 있습니다.
3. 모델 체크포인트 저장하기: 매번 저장할 로직을 짜고 `torch.save`를 호출하는 것도 귀찮습니다. `pytorch_lightning.callbacks.ModelCheckpoint`라는 콜백을 사용하면 이를 간편하게 작업할 수 있습니다.
4. 학습 조기종료 로직: 이 로직도 구현하는게 매번 번거로운데 이 또한 `pytorch_lightning.callbacks.EarlyStopping`을 통해서 쉽게 달성할 수 있습니다.

In [13]:
class MLPClassifier(pl.LightningModule):
    def __init__(self,
                 input_dim=28*28,
                 hidden_dims=(128, 64),
                 dropout=0.2,
                 num_classes=10,
                 lr=1e-3):
        super().__init__()

        h1, h2 = hidden_dims
        self.model = nn.Sequential(
            nn.Flatten(),  # 입력 이미지(28x28)를 1D 벡터로 변환
            nn.Linear(input_dim, h1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h2, num_classes)
        )

        # 한 epoch 내에서 기록되는 값들을 저장하는 리스트입니다.
        self.training_step_outputs = []
        self.validation_step_outputs = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # 정답코드
        # 위에서 제작하신 함수를 그대로 사용해주세요
        # 반환할 결과값을 `self.training_step_outputs`에 append 해주세요
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        result = {'loss': loss, 'acc': acc}
        self.log_dict(result, prog_bar=True)
        self.training_step_outputs.append(result)
        return loss

    def on_train_epoch_end(self):
        # 정답코드: train epoch의 평균 Loss와 평균 정확도를 계산해주세요.
        # 마지막에 validation_step_outputs를 clear해주세요
        avg_loss = torch.stack([out['loss'] for out in self.training_step_outputs]).mean()
        avg_acc  = torch.stack([out['acc'] for out in self.training_step_outputs]).mean()
        self.log_dict({'epoch_train_loss': avg_loss,
                       'epoch_train_acc': avg_acc}, prog_bar=True)
        self.training_step_outputs.clear()

    def validation_step(self, batch, batch_idx):
        # 정답코드
        # 위에서 제작하신 함수를 그대로 사용해주세요
        # 반환할 결과값을 `self.validation_step_outputs`에 append 해주세요
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        result = {'val_loss': loss, 'val_acc': acc}
        self.log_dict(result, prog_bar=True)
        self.validation_step_outputs.append(result)
        return result

    def on_validation_epoch_end(self):
        # 정답코드: validation epoch의 평균 Loss와 평균 정확도를 계산해주세요.
        # 마지막에 validation_step_outputs를 clear해주세요
        avg_loss = torch.stack([out['val_loss'] for out in self.validation_step_outputs]).mean()
        avg_acc  = torch.stack([out['val_acc'] for out in self.validation_step_outputs]).mean()
        self.log_dict({'epoch_val_loss': avg_loss,
                       'epoch_val_acc': avg_acc}, prog_bar=True)
        self.validation_step_outputs.clear()

    def predict_step(self, batch, batch_idx):
        # TODO: `trainer.predict`를 사용하기 위한 `predict_step을 구현해봅시다.
        # 결과를 한데 모으기 위해 전체 logits를 계산해봅시다. 반환값으로 `logits`를 key로 하도록 dict에 넣어주세요.
        # 추가로 라벨값을 같이 받아 `y_true`를 key로 하도록 dict에 넣어주세요.
        # 참고: https://lightning.ai/docs/pytorch/LTS/common/lightning_module.html#prediction-loop
        x, y = batch
        logits = self(x)
        return dict(logits=logits, y_true=y)

    def configure_optimizers(self):
        # 정답코드
        # 위에서 제작하신 함수를 그대로 사용해주세요
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [14]:
# 정답코드
# W&B 로거를 생성해주세요.
# 계정 생성 후 API Key를 입력해주어야합니다.
# 번거로우시다면 패스해도됩니다.
from pytorch_lightning.loggers import WandbLogger
# wandb_logger = WandbLogger(project='FashionMNIST-PL', log_model=True)

# 정답코드
# 체크포인트를 자동으로 생성해주는 콜백함수를 정의해주세요.
# API Reference를 보고 어떤 인자가 들어가야하는지 확인해주세요.
# https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.callbacks.ModelCheckpoint.html
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_cb = ModelCheckpoint(
    monitor='epoch_val_loss',              # 모니터링할 metric
    save_top_k=1,                          # 가장 좋은 1개만 저장
    mode='min',                            # 값이 작을수록 우수
    filename='best-{epoch:02d}-{epoch_val_loss:.4f}'  # 저장 파일명 패턴
)

# 정답코드
# 학습의 조기종료를 자동으로 생성해주는 콜백함수를 정의해주세요.
# API Reference를 보고 어떤 인자가 들어가야하는지 확인해주세요.
# https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.callbacks.EarlyStopping.html
from pytorch_lightning.callbacks import EarlyStopping
earlystop_cb = EarlyStopping(
    monitor='epoch_val_loss',  # 모니터링할 metric
    patience=3,                # 개선이 없을 때 중단까지 대기할 epoch 수
    mode='min'                 # 손실이 낮을수록 좋음
)

# 정답코드 Trainer를 통한 학습
# callbacks와 logger는 다음처럼 넣어주면 됩니다.
"""
요구사항
- max_epochs를 20으로 늘려주세요
- gradient accumulation을 자유롭게 설정해주세요.
- gradient clipping을 설정해주세요 (1.0)
"""
trainer = pl.Trainer(
    max_epochs=20,                     # 전체 학습 epoch 수
    accelerator='auto',                # CPU/GPU 자동 선택
    accumulate_grad_batches=4,         # Gradient Accumulation
    gradient_clip_val=1.0,             # Gradient Clipping (폭주 방지) -> 기울기의 L2 norm이 1.0을 넘으면 비율로 줄임
    # logger=wandb_logger,             # W&B 로깅 (선택)
    callbacks=[checkpoint_cb, earlystop_cb], # 특정 시점에 자동으로 호출되는 함수
)


# 데이터모듈, 모델 선언
dm = FashionMNISTDataModule(batch_size=64)
model = MLPClassifier()

# 학습
trainer.fit(model, dm)

# Validation Dataset에 대해 검증
trainer.validate(model, dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 109 K  | train
---------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.438     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       epoch_val_acc       │    0.8860469460487366     │
│      epoch_val_loss       │    0.32205653190612793    │
│          val_acc          │    0.8855000138282776     │
│         val_loss          │    0.32323360443115234    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.32323360443115234,
  'val_acc': 0.8855000138282776,
  'epoch_val_loss': 0.32205653190612793,
  'epoch_val_acc': 0.8860469460487366}]

In [15]:
# 전체 결과를 추론한 후 확인해봅시다.
predictions = trainer.predict(model, datamodule=dm, return_predictions=True)

import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report

# 전체 logits과 y_true 합치기
all_logits = torch.cat([p['logits'] for p in predictions], dim=0)
all_labels = torch.cat([p['y_true'] for p in predictions], dim=0)

# 예측 라벨 계산
all_probs = F.softmax(all_logits, dim=1)
all_preds = all_probs.argmax(dim=1)

num_classes = all_logits.shape[1]
class_names = [f"class_{i}" for i in range(num_classes)]

# ===== 정확도 =====
accuracy = (all_preds == all_labels).float().mean().item()
print(f"전체 정확도: {accuracy:.2%}\n")

# ===== 클래스별 분포 =====
print("=== 클래스별 정답/예측 분포 ===")
for i, name in enumerate(class_names):
    true_count = (all_labels == i).sum().item()
    pred_count = (all_preds == i).sum().item()
    print(f"{name}: 실제 {true_count}개 / 예측 {pred_count}개")

# ===== 혼동 행렬 =====
cm = confusion_matrix(all_labels.numpy(), all_preds.numpy())
print("\n=== 혼동 행렬 ===")
print(cm)

# ===== 정밀도, 재현율, F1 =====
print("\n=== Classification Report ===")
print(classification_report(all_labels.numpy(), all_preds.numpy(), target_names=class_names))

Predicting: |          | 0/? [00:00<?, ?it/s]

전체 정확도: 88.55%

=== 클래스별 정답/예측 분포 ===
class_0: 실제 1000개 / 예측 932개
class_1: 실제 1000개 / 예측 975개
class_2: 실제 1000개 / 예측 912개
class_3: 실제 1000개 / 예측 1115개
class_4: 실제 1000개 / 예측 1034개
class_5: 실제 1000개 / 예측 970개
class_6: 실제 1000개 / 예측 1024개
class_7: 실제 1000개 / 예측 1024개
class_8: 실제 1000개 / 예측 1002개
class_9: 실제 1000개 / 예측 1012개

=== 혼동 행렬 ===
[[804   0  11  33   3   2 138   0   9   0]
 [  4 966   0  25   3   0   1   0   1   0]
 [ 14   1 771  21 109   1  82   0   1   0]
 [ 14   7   3 935   9   0  28   0   4   0]
 [  0   1  54  56 834   0  53   0   2   0]
 [  0   0   0   1   0 943   0  36   2  18]
 [ 92   0  72  38  75   0 715   0   8   0]
 [  0   0   0   0   0  16   0 951   1  32]
 [  3   0   1   6   1   3   7   5 974   0]
 [  1   0   0   0   0   5   0  32   0 962]]

=== Classification Report ===
              precision    recall  f1-score   support

     class_0       0.86      0.80      0.83      1000
     class_1       0.99      0.97      0.98      1000
     class_2       0.85      0.77   

# 마치며...

오늘은 `scikit-learn`과 `pytorch-lightning`을 한 번 다시 톺아보는 시간을 가졌습니다. 헷갈리는 부분이 있으면 다시 API Reference를 확인해보며 다져봅시다. 위에서 확인한 학습 결과가 좋지 않으면, hyperparameter를 튜닝하면서 조절해봅시다.

## Further Readings

- Understanding the difficulty of training deep feedforward neural networks: MLP 학습 초반 기울기 소실/폭주 문제와 각종 초기화 기법(Xavier, He 등)의 이론적 배경을 깊이 있게 다룹니다.
  - https://proceedings.mlr.press/v9/glorot10a.html

- Feature Visualization: How Neural Networks See the World: 은닉층 뉴런이 학습 중 어떤 특징을 포착하는지 시각화 기법을 통해 분석하며, MLP 내부 표현 학습 과정을 직관적으로 이해하게 해줍니다.
  - https://distill.pub/2017/feature-visualization/

## Open-Ended Question

- **초기화 전략 비교 분석**: MLP에 Xavier, He, Orthogonal 초기화 기법을 각각 적용해 수렴 속도 및 최종 정확도 차이를 실험하고, 학습 곡선을 시각화해보세요.

- **하이퍼파라미터 자동 탐색**: Grid Search나 Bayesian Optimization으로 은닉층 개수, 뉴런 수, 학습률, 배치 크기 등을 자동으로 최적화하여 성능이 가장 뛰어난 설정을 찾아 보세요.